# Setup

## Import Packages

In [1]:
import os, json
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.chains import LLMChain

from langchain.agents import AgentExecutor, AgentOutputParser, initialize_agent
from custom_agents import HumanSearchRetrievalToolkit, RetrievalToolkit, HumanRetrievalToolkit

## Setup Directories

In [2]:
MAIN_DIR = ".."
EMB_DIR = os.path.join(MAIN_DIR, "data", "emb_store", "uc", "faiss", "text-embedding-ada-002", "v8-add-tables_2500_500")

with open(os.path.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    api_keys = json.load(f)
    
os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]

## Load Test Cases

In [3]:
with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all.txt"),
          "r", encoding="utf-8-sig") as f:
    test_cases = f.readlines()
test_cases = [test_case.rstrip() for test_case in test_cases]

print("Total number of test cases:", len(test_cases))

Total number of test cases: 30


# ReAct and MRKL Agents

## Define Toolkit

In [4]:
docsearch = FAISS.load_local(EMB_DIR, OpenAIEmbeddings())
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [5]:
toolkit1 = HumanSearchRetrievalToolkit(
    serper_api_key=api_keys["SERP_API"],
    llm = llm, docsearch=docsearch
)

tools = toolkit1.get_tools()
for idx, tool in enumerate(tools):
    print(f"Tool number {idx+1}: {tool.name}")

Tool number 1: human
Tool number 2: google_serper
Tool number 3: Docsearch QA Tool
Tool number 4: General Knowledge


## Define Agent Instructions

In [6]:
PREFIX = """You are a helpful and honest physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
If you do not know the answer, you will not make up an answer and instead ask further questions for clarifications

Your job is to ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

SUFFIX = """Begin!

Patient Profile: {input}
Thought: {agent_scratchpad}"""

AGENT_PROMPT_1 = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,    
)

print(AGENT_PROMPT_1.template)

You are a helpful and honest physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
If you do not know the answer, you will not make up an answer and instead ask further questions for clarifications

Your job is to ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

You have access to the following tools:

human: You can ask a human for guidance when you think you got stuck or you are not sure what to do next. The input should be a question for the human.
google_serper: A low-cost Google Search API.Useful for when you need to answer questions about current

## Setup Agent and AgentExecutor

In [7]:
agent_llm = LLMChain(
    llm = ChatOpenAI(model_name = "gpt-4", temperature = 0, max_tokens = 512),
    prompt = AGENT_PROMPT_1
)
REACT_AGENT_1 = ZeroShotAgent(llm_chain=agent_llm, allowed_tools = [tool.name for tool in tools])

agent_args = {"max_iterations": 3, "early_stopping_method": "generate"}
agent_executor = AgentExecutor.from_agent_and_tools(agent=REACT_AGENT_1, tools=tools, verbose=True, **agent_args)

## Run Test Case

In [8]:
sample_case = test_cases[0]
agent_executor(sample_case)



> Entering new AgentExecutor chain...


The patient is newly diagnosed with moderate UC and has extraintestinal manifestations. I need to consider these factors while suggesting the biological drugs. I will use the Docsearch QA Tool to find the best treatment options for a newly diagnosed patient with these conditions.
Action: Docsearch QA Tool
Action Input: Best biological drugs for newly diagnosed moderate UC with extraintestinal manifestations

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: {'question': 'Best biological drugs for newly diagnosed moderate UC with extraintestinal manifestations', 'answer': "The best biological drugs for newly diagnosed moderate Ulcerative Colitis (UC) with extraintestinal manifestations are Infliximab, Vedolizumab, and Ustekinumab. Infliximab is ranked highest for induction of clinical remission and endoscopic improvement in biologic-naïve patients. Ustekinumab and Tofacitinib are ranked highest in patients with prior exposure to TNF antagonists. Upadac

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..



> Finished chain.

Observation: {'question': 'Pros and cons of Infliximab and Vedolizumab for treatment of moderate UC with extraintestinal manifestations.', 'answer': "Infliximab and Vedolizumab are both used for the treatment of moderate Ulcerative Colitis (UC). Infliximab has been shown to have a higher rate of steroid-free clinical remission and endoscopic remission compared to other treatments in UC patients. It is also considered the most effective for induction in biologic-naïve patients with moderate to severely active Crohn's Disease (CD) (", 'sources': 'data/document_store/uc/juillerat 2022.pdf, data/document_store/uc/PIIS1542356520300446.pdf). \n\nVedolizumab, on the other hand, has been shown to have a more persistent clinical response and steroid-free remission at one year in UC patients compared to anti-TNF agents (SOURCE: data/document_store/uc/juillerat 2022.pdf). However, an increased incidence of extraintestinal manifestations has been observed in vedolizumab-treated

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens p

RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.